In [72]:
class MazeObject:
    def __init__(self):
        self.state = None  # State represents the direction of current flow (e.g., "U-D")
        self.faces = {'U': False, 'D': False, 'R': False, 'L': False}  # LEDs for each face

    def set_state(self, state):
        """Set the state and update the LEDs based on the current flow."""
        self.state = state
        self._update_faces()

    def _update_faces(self):
        """Update the LEDs based on the current flow direction."""
        # Reset all LEDs
        self.faces = {'U': False, 'D': False, 'R': False, 'L': False}

        if self.state:
            # Split the state into source and destination faces
            source, dest = self.state.split('-')
            if source in self.faces and dest in self.faces:
                self.faces[source] = True  # LED at source face lights up
                self.faces[dest] = True   # LED at destination face lights up

    def __repr__(self):
        return f"MazeObject(state={self.state}, faces={self.faces})"

In [83]:
from collections import deque

class Maze:
    def __init__(self):
        self.grid = [[MazeObject() for _ in range(5)] for _ in range(5)]
        self.entrance = None
        self.exit = None
        self.walls = set()  # Store walls as frozensets of tuples

        # Initialize border faces as False
        self._initialize_borders()

    def _initialize_borders(self):
        """Set external faces of border objects to False."""
        for x in range(5):
            for y in range(5):
                if x == 0:  # Left border
                    self.grid[x][y].faces['L'] = False
                if x == 4:  # Right border
                    self.grid[x][y].faces['R'] = False
                if y == 0:  # Top border
                    self.grid[x][y].faces['U'] = False
                if y == 4:  # Bottom border
                    self.grid[x][y].faces['D'] = False

    def set_entrance(self, x, y):
        self.entrance = (x, y)

    def set_exit(self, x, y):
        self.exit = (x, y)

    def add_wall(self, x1, y1, x2, y2):
        """Add a wall between two adjacent objects."""
        if abs(x1 - x2) + abs(y1 - y2) != 1:
            raise ValueError("Walls can only be added between adjacent objects.")

        # Store the wall as a frozenset to avoid duplicates
        wall = frozenset({(x1, y1), (x2, y2)})
        self.walls.add(wall)

        # Update the faces of the objects to reflect the wall
        if x1 == x2:  # Vertical wall (U-D connection)
            if y1 < y2:
                self.grid[x1][y1].faces['D'] = False
                self.grid[x2][y2].faces['U'] = False
            else:
                self.grid[x1][y1].faces['U'] = False
                self.grid[x2][y2].faces['D'] = False
        elif y1 == y2:  # Horizontal wall (L-R connection)
            if x1 < x2:
                self.grid[x1][y1].faces['R'] = False
                self.grid[x2][y2].faces['L'] = False
            else:
                self.grid[x1][y1].faces['L'] = False
                self.grid[x2][y2].faces['R'] = False

    def propagate_potential(self):
        """Propagate the potential from the entrance to the exit."""
        if not self.entrance or not self.exit:
            print("Entrance and exit must be set.")
            return

        # Initialize BFS queue
        queue = deque()
        entrance_x, entrance_y = self.entrance
        queue.append((entrance_x, entrance_y, None, None))  # (x, y, parent, direction)

        # Track visited objects, their parents, and the direction of movement
        visited = {}
        visited[(entrance_x, entrance_y)] = (None, None)  # (parent, direction)

        # Perform BFS
        while queue:
            x, y, parent, direction = queue.popleft()

            if (x, y) == self.exit:
                print("Potential reached the exit!")
                break

            # Explore adjacent objects
            for dir, (dx, dy) in zip(['L', 'R', 'U', 'D'], [(-1, 0), (1, 0), (0, -1), (0, 1)]):
                nx, ny = x + dx, y + dy
                if 0 <= nx < 5 and 0 <= ny < 5:
                    if (nx, ny) not in visited and not self._is_wall_between((x, y), (nx, ny)):
                        visited[(nx, ny)] = ((x, y), dir)  # Mark as visited with parent and direction
                        queue.append((nx, ny, (x, y), dir))

        # Reconstruct the valid path and exit face sequence
        if self.exit in visited:
            self._mark_valid_path(visited)
            self._display_exit_face_sequence(visited)
        else:
            print("No valid path to the exit!")

    def _is_wall_between(self, pos1, pos2):
        """Check if there is a wall between two positions."""
        return frozenset({pos1, pos2}) in self.walls

    def _get_opposite_dir(self, dir):
        """Get the opposite direction."""
        if dir == 'U':
            return 'D'
        elif dir == 'D':
            return 'U'
        elif dir == 'L':
            return 'R'
        elif dir == 'R':
            return 'L'

    def _mark_valid_path(self, visited):
        """Mark the valid path from the exit back to the entrance."""
        current = self.exit
        while current:
            x, y = current
            self.grid[x][y].state = "VALID"  # Mark as part of the valid path
            current, _ = visited[(x, y)]  # Move to parent

    def _display_exit_face_sequence(self, visited):
        """Display the sequence of exit faces from the entrance to the exit."""
        current = self.exit
        path = []
        while current:
            x, y = current
            parent, direction = visited[(x, y)]
            if direction:
                # The exit face is the opposite of the direction of movement
                #exit_face = self._get_opposite_dir(direction)
                #print( parent, direction, current)
                path.append(direction)  # Add the exit face to the path
            current = parent

        # Reverse the path to show from entrance to exit
        #print(path)
        path.reverse()
        #print(path)
        print("Exit face sequence:", "-".join(path))

    def visualize(self):
        """Visualize the maze as a 5x5 grid, showing the valid path with 'X'."""
        for y in range(5):
            for x in range(5):
                if self.grid[x][y].state == "VALID":  # If the object is part of the valid path
                    print("X", end=" ")
                else:
                    print(".", end=" ")
            print()  # Newline after each row

In [84]:
maze = Maze()
maze.set_entrance(0, 0)
maze.set_exit(4, 4)

# Add walls
maze.add_wall(0, 0, 0, 1)  
maze.add_wall(1, 0, 1, 1) 
maze.add_wall(3, 0, 3, 1)  
maze.add_wall(4, 0, 4, 1)  
maze.add_wall(1, 1, 2, 1)  
maze.add_wall(2, 1, 2, 2)  
maze.add_wall(4, 1, 4, 2)  

maze.add_wall(1, 2, 1, 3)  
maze.add_wall(2, 2, 2, 3)  
maze.add_wall(3, 2, 3, 3)  
maze.add_wall(3, 2, 4, 2)  

maze.add_wall(0, 3, 0, 4)  
maze.add_wall(1, 3, 1, 4) 
maze.add_wall(2, 3, 2, 4) 
maze.add_wall(3, 4, 4, 4) 
 

# Propagate the potential
maze.propagate_potential()

# Visualize the maze
maze.visualize()

Potential reached the exit!
Exit face sequence: R-R-D-R-D-L-L-L-D-R-R-R-R-D
X X X . . 
. . X X . 
X X X X . 
X X X X X 
. . . . X 
